In [23]:
import numpy as np
import cv2 as cv

In [24]:
# Function to read the secret image
def read_image(image_name):
    image = cv.imread(image_name)
    print("The shape of the secret image is = ", image.shape)
    resized_image = cv.resize(image, (500,500))
    cv.imshow("Secret Image",resized_image)
    cv.waitKey(0)
    return image

In [25]:
# Function to generate a random key
def generateRandomKey(key_shape):
    dealer_key = np.zeros(key_shape).astype(int)
    for i in range(0, len(dealer_key)):
        for j in range(0, len(dealer_key[i])):
            for k in range(0, 3):
                dealer_key[i][j][k] = (np.random.randint(0,255))
    return dealer_key


In [26]:
# Function to convert the secret image into encrypted image
def convertToEncryptedImage(image, random_key):
    print("Converting to Encrypted Image...")
    encrypted_image = np.zeros(image.shape).astype(int)
    
    for i in range(0, len(random_key)):
        for j in range(0, len(random_key[i])):
            for k in range(0, 3):
                encrypted_image[i][j][k] = image[i][j][k]^random_key[i][j][k]
    
    print("The shape of encrypted image = " ,encrypted_image.shape)
    return encrypted_image

In [28]:
N = input('Enter the total number of shares (N) : ')
K = input("Enter the number of shares required to retreive the seccret image (K) :")
image_name = input("Enter the name of the image (with extension)")
image = read_image(image_name)
random_key = generateRandomKey(image.shape)
encrypted_image = convertToEncryptedImage(image, random_key)

The shape of the secret image is =  (1225, 840, 3)
Converting to Encrypted Image...
The shape of encrypted image =  (1225, 840, 3)
